In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import mysql.connector
import time

In [2]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="jobs-scraping.cux1s0fa60hj.us-east-2.rds.amazonaws.com",
    user="root",
    password="123456789",
    database="job_scraping_database"
)

cursor = mydb.cursor()


In [3]:
# Define chrome_options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=./chromedriver.exe")

In [4]:
wd = webdriver.Chrome(options = chrome_options)
time.sleep(1)
wd.get("https://www.simplyhired.com/search?q=entry+level+marketing&l=United+States")
time.sleep(3)
# # Simulate a click anywhere on the screen
# action_chains = ActionChains(wd)
# action_chains.move_by_offset(100, 100).click().perform()

In [5]:
page_number = 1
next_page_list = []
def simplyhired_scraper():
    global page_number 
    
    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("li", class_ = "css-0")
    
    for job in jobs:
        #JobId
        job_id = job.find("div", {"data-testid": "searchSerpJob"})["data-jobkey"]
        
        #Check if the JobId already exists in the table
        select_query = "SELECT JobId FROM marketing_jobs WHERE JobId = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()
        
        if result is None:
            #Job Title
            job_title = job.find("h3", {"data-testid": "searchSerpJobTitle"}).text.strip()
            
            #Job Link
            job_link = "https://www.simplyhired.com" + job.find("h3", {"data-testid": "searchSerpJobTitle"}).find("a")["href"]
            
            #Company Name
            company_name_elem = job.find("span", {"data-testid": "companyName"})
            company_name = company_name_elem.text.strip() if company_name_elem else ""

            
            #Location
            location = job.find("span", {"data-testid": "searchSerpJobLocation"}).text.strip()
            
            driver = webdriver.Chrome(options = chrome_options)
            time.sleep(1)
            driver.get(job_link)
            time.sleep(3)
            job_src = driver.page_source
            job_soup = bs(job_src, "html.parser")
            
            #Salary
            salary_elem = job_soup.find("span", {"data-testid": "viewJobBodyJobCompensation"})
            salary = salary_elem.text.strip() if salary_elem else ""
            
            #Job Description
            job_des_elem = job_soup.find("div", {"data-testid": "viewJobBodyJobFullDescriptionContent"})
            job_des = job_des_elem.text.strip() if job_des_elem else ""
            
            #Skill
            list_skill = ""
            skill_elem = job_soup.find("div", {"data-testid": "viewJobQualificationsContainer"})
            skill_list = []
            if skill_elem:
                skills = skill_elem.find_all("li")
                for skill in skills:
                    skill_list.append(skill.text.strip())
                list_skill = ", ".join(skill_list)
            else:
                list_skill = ""
            
            #Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO marketing_jobs (JobId, Title, Company, Location, Apply, Salary, JobDescription, Skills)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, salary, job_des, list_skill)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
            driver.quit()
        else:
            print("Skipping duplicate JobId:", job_id)
    print("Data updated for page:", page_number)
    page_number += 1
    try:
        next_button = wd.find_element(By.XPATH, f"//a[@data-testid = 'paginationBlock{page_number}']")
        next_button.click()
        time.sleep(3)
        simplyhired_scraper()
    except NoSuchElementException:
        cursor.close()
        mydb.close()
        wd.quit()

simplyhired_scraper()

Skipping duplicate JobId: pULOuxqfkaCSLW-YNcBtWfQVeBUgLl8frux4FsOLnqV3Y7U1m2beVw
Skipping duplicate JobId: P07FLbumxCBVEjkHzWj-CIHfxe13tYhdSCjIDgvmUSVXTEM8Y-cjnA
Skipping duplicate JobId: 588XPFT_ZkUqmYKu20YCm8UrsAy1po598KdNZ8qRYubtj8SeFBFzig
Skipping duplicate JobId: eUQbctaicEKFrXPLFYtS_3mcAXb7vMEGoVk13Jz6i2ZdDQA2Dg-o0Q
Skipping duplicate JobId: 0GpMW8ziD9TK7iYRnFR1f3fctB6nbgPc6kFJlFl49BP9uXh9fhK25A
Skipping duplicate JobId: 8phr7D0xiIVTUptsVJsftq9zybPrYN_WGbydeyJ5jH8CnUntuec0oQ
Skipping duplicate JobId: xoGaltSXT7ZphZUhKlW24e5dwXSPSqlFTDZUXnC-Tq7O3JRRvI9IfA
Skipping duplicate JobId: OsC_iq8H0jgvAZssaiEYPXP3q894avV0FDQX5R3ptjA9A4RV0Bm75Q
Skipping duplicate JobId: PjnJ1PddFPi4y4e30IVpWyizntDTPtehTTLapKHbdBSwMKfxQ_1OtQ
Skipping duplicate JobId: N_oGVpd4UILGu0n0uxxsrXTBAOD3xm8Tk4e72aTVlwlTHAhgZaOVLQ
Skipping duplicate JobId: -r9Y8VO3b0wstuLVHpF0rThrirXb4M1bHDiSFsuF23VynkFwXr5jyA
Skipping duplicate JobId: VZptYtTA6s-lnnO2bGnCqPBvJOOPffopt72Kp8PfStaJdV5qYzj0IA
Skipping duplicate JobId: wA